In [28]:
import cv2
import pickle
import os
import time
import requests
import threading
from ultralytics import YOLO
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from threading import Thread

In [35]:
# ตั้งค่า LINE Notify
LINE_NOTIFY_TOKEN = "wDWc4yjjTH90dosYSXKva3tfhzW5Qz38NHgGHgt8HfL"
LINE_NOTIFY_URL = "https://notify-api.line.me/api/notify"


In [37]:
# Function to send LINE Notify message with image
def send_line_notify(message, image_path=None):
    headers = {"Authorization": f"Bearer {LINE_NOTIFY_TOKEN}"}
    data = {"message": message}

    if image_path:
        files = {"imageFile": open(image_path, "rb")}
        requests.post(LINE_NOTIFY_URL, headers=headers, data=data, files=files)
        files["imageFile"].close()
    else:
        requests.post(LINE_NOTIFY_URL, headers=headers, data=data)

# โหลดโมเดล YOLO
model = YOLO(r"runs/detect/train/weights/best.pt")

# โหลดวิดีโอ
video_path = "test_video/fire_test1.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Failed to load video!")
    exit()

# โหลด Ground Truth ที่เคยบันทึกไว้
if os.path.exists("ground_truth\ground_truth1.pkl"):
    with open("ground_truth\ground_truth1.pkl", "rb") as f:
        ground_truth = pickle.load(f)
else:
    print("❌ ไม่พบไฟล์ Ground Truth! กรุณารัน 'collect_ground_truth.py' ก่อน")
    exit()

predictions = []  # เก็บค่าที่โมเดลคาดการณ์
frame_index = 0   # ใช้ในการอ้างอิง Ground Truth

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # วิดีโอจบ

    results = model(frame)
    fire_in_frame = False  # เริ่มต้นไม่มีไฟ

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0].item()
            label = result.names[int(box.cls[0])]

            if "fire" in label.lower() and conf > 0.50:
                fire_in_frame = True  # มีไฟในเฟรมนี้
                center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
                box_size = (x2 - x1) * (y2 - y1)

                # 🔥 ระบุตำแหน่งไฟ
                horizontal_pos = "Left" if center_x < frame_width * 0.33 else "Right" if center_x > frame_width * 0.66 else "Center"
                vertical_pos = "Top" if center_y < frame_height * 0.33 else "Bottom" if center_y > frame_height * 0.66 else "Middle"
                depth_pos = "Near" if box_size > (frame_width * frame_height * 0.05) else "Far" if box_size < (frame_width * frame_height * 0.02) else "Mid-range"

                fire_location = f"{horizontal_pos}, {vertical_pos}, {depth_pos}"
                print(f"🚨 Fire detected at: {fire_location}")

                # วาดกล่องรอบไฟ
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.putText(frame, fire_location, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

                # Save the frame with fire detection
                img_filename = "fire_detected_frame.jpg"
                cv2.imwrite(img_filename, frame)

                # Send LINE notification with image
                alert_message = f"🔥 Fire detected at: {fire_location}"
                Thread(target=send_line_notify, args=(alert_message, img_filename)).start()

    predictions.append(1 if fire_in_frame else 0)

    # แสดงผลวิดีโอ
    frame_resized = cv2.resize(frame, (1280, 720))
    cv2.imshow("🔥 Fire Detection System", frame_resized)

    # กด 'q' เพื่อออก
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

    frame_index += 1

cap.release()
cv2.destroyAllWindows()

# คำนวณค่าความแม่นยำ (Precision, Recall, F1-score, Accuracy)
if len(ground_truth) <= len(predictions):  # ตรวจสอบว่า ground_truth น้อยกว่าหรือเท่ากับ predictions
    min_length = min(len(ground_truth), len(predictions))  # เลือกขนาดที่น้อยกว่า

    # ตัด ground_truth และ predictions ให้มีขนาดเท่ากัน
    ground_truth = ground_truth[:min_length]
    predictions = predictions[:min_length]

    # คำนวณค่าความแม่นยำ
    recall = recall_score(ground_truth, predictions)
    f1 = f1_score(ground_truth, predictions)
    accuracy = accuracy_score(ground_truth, predictions)
    precision = precision_score(ground_truth, predictions)

    print(f"\n📊 **Accuracy Metrics**")
    print(f"✅ Precision: {precision:.4f}")
    print(f"✅ Recall: {recall:.4f}")
    print(f"✅ F1 Score: {f1:.4f}")
    print(f"✅ Accuracy: {accuracy:.4f}")

else:
    print("⚠️ จำนวนเฟรมของ Ground Truth น้อยกว่าผลลัพธ์ที่ได้ กรุณาเก็บ Ground Truth ใหม่!")  


0: 384x640 1 fire, 10.2ms
Speed: 2.4ms preprocess, 10.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 fire, 12.5ms
Speed: 2.0ms preprocess, 12.5ms inference, 7.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 fire, 13.4ms
Speed: 2.0ms preprocess, 13.4ms inference, 6.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 fire, 11.3ms
Speed: 3.5ms preprocess, 11.3ms inference, 4.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 fire, 12.7ms
Speed: 2.0ms preprocess, 12.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 fire, 12.4ms
Speed: 2.1ms preprocess, 12.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 fire, 8.0ms
Speed: 3.1ms preprocess, 8.0ms inference, 3.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 fire, 12.1ms
Speed: 1.8ms preprocess, 12.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x64